# Exercise 1 — KSW Solution

Build on the **demo** by exploring: (A) generation length, (B) numerical precision, and (C) KV cache.

## 0) Environment guard

#### Built on top of the Demo
Avoid NumPy 2.x + TensorFlow wheel conflicts by disabling optional TF/Flax imports in `transformers`.

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

## 1) Imports & globals

#### Built on top of the Demo
If the preferred model isn't available, we fall back to a tiny model to keep the notebook runnable everywhere.

In [ ]:
import time
from typing import Dict, Any, List
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device={DEVICE}; NumPy={np.__version__}; Torch={torch.__version__}")

PREFERRED_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

DEFAULT_PROMPT = (
        "Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural "
        "intelligence displayed by humans and animals. Leading AI textbooks define the field as the study "
        "of intelligent agents: any system that perceives its environment and takes actions that maximize "
        "its chance of achieving its goals. Colloquially, the term \"artificial intelligence\" is often "
        "used to describe machines that mimic cognitive functions that humans associate with the human mind, "
        "such as learning and problem-solving."
)


## 2) Helpers 

#### Built on top of the Demo
**Todo #1: Resolve Requested Numeric Precision** \
In the _resolve_dtype function
- write the logic to convert a user-provided precision string ("float16" or "float32") into the corresponding torch.dtype.
- if the user provided string is neither float16 nor float32 , raise ValueError with a clear message.
- Your implementation must include a CPU guard: if the device is CPU and the user requests float16, the function should fall back to torch.float32 and return a descriptive label like "float32 (forced on CPU)". This is because most CPUs don’t support fp16 efficiently.

In [ ]:
def _resolve_dtype(dtype_str: str, device: torch.device):
    """
      1) Normalize/validate `dtype_str` (accept 'float16' or 'float32'; case-insensitive is fine).
         - If unsupported, raise ValueError with a clear message.

      2) CPU guard:
         - If device.type == 'cpu' and requested dtype is float16, fall back to torch.float32
           (most CPUs don’t support fp16 efficiently).
         - Return both: (torch.float32, "float32 (forced on CPU)").

      3) Otherwise map the string to the torch dtype:
         - 'float16' -> torch.float16
         - 'float32' -> torch.float32

      4) Return a tuple: (resolved_torch_dtype, label_string), where label_string is
         a human-readable descriptor like 'float16' or 'float32 (forced on CPU)'.
    """
    if not isinstance(dtype_str, str):
        raise ValueError(f"dtype_str must be a string, got {type(dtype_str).__name__}")


    # Normalize
    normalized_dtype = dtype_str.strip().lower()

    # Check and Return
    if device.type == 'cpu' and dtype == 'float16':
        normalized_dtype = 'float32 (forced on CPU)'
        return torch.float32, normalized_dtype

    if dtype == 'float16':
        return torch.float16, normalized_dtype
    elif dtype == 'float32':
        return torch.float32, normalized_dtype
    else:
        raise ValueError(f"Unsupported dtype '{dtype_str}'. Supported values are: 'float16' or 'float32'.")

    return 

## TODO 2: Load the Model and Tokenizer: 
Implement the load_model_and_tokenizer function. 
- This function should first call your _resolve_dtype helper to get the correct data type.
- Then, use AutoTokenizer.from_pretrained and AutoModelForCausalLM.from_pretrained to load the assets.
- Ensure the model is moved to the correct device, set to evaluation mode with .eval(), and that the tokenizer's pad_token is set if necessary to avoid warnings.def load_model_and_tokenizer(model_name: str, dtype_str: str = "float16"):

## Code Notes

1️⃣ low_cpu_mem_usage=True

**What it means**
```
Load the model without ever holding a full copy in CPU RAM.
```
Normally, from_pretrained():
- Downloads weights to disk
- Loads them fully into CPU memory
- Copies them to GPU

For big models, step (2) can blow up your RAM.

**With low_cpu_mem_usage=True:**
- Weights are streamed
- Layers are materialized one by one
- Much lower peak RAM usage

**When to use it**
- Large models
- Limited system RAM
- Servers / containers / CI

**Tradeoffs**
- Slightly slower load time
- Requires Hugging Face 🤗 accelerate under the hood

2️⃣ trust_remote_code=False (default, safe)
Meaning
```
“If this repo requires custom Python code, do NOT run it.”
```

When you do:
```
AutoModelForCausalLM.from_pretrained("some-model")
```

Transformers normally:
- Downloads weights (.bin, .safetensors)
- Instantiates a built-in model class (e.g. LlamaForCausalLM)
- Loads the weights into that class
👉 No custom code runs.

**What changes when a repo has “remote code”**

Some model repos include Python files like:
- modeling_my_model.py
- configuration_my_model.py
- generation_utils.py


These files can:
- Define new model classes
- Override forward()
- Change attention, KV cache, generation, etc.
- Run arbitrary Python at import time

That’s what Hugging Face calls remote code.

In [6]:
def load_model_and_tokenizer(model_names: list[str], dtype_str: str = "float16"):
    """
      1) Resolve the requested precision:
         - Call `_resolve_dtype(dtype_str, device)` to get `(torch_dtype, dtype_label)`.
         - Ensure CPU requests for 'float16' are coerced to float32.

      2) Build a candidate list of model ids:
         - Start with `[model_name]`.
         - (Optional) Append `FALLBACK_MODELS` so the function still works if the preferred model
           isn’t available in the environment.

      3) Try candidates in order:
         for name in candidates:
           try:
             - tokenizer = AutoTokenizer.from_pretrained(name, use_fast=True)
             - model = AutoModelForCausalLM.from_pretrained(name, torch_dtype=torch_dtype)
             - Move model to `device` and call `model.eval()`
             - If `tokenizer.pad_token_id` is None and `tokenizer.eos_token_id` exists,
               set `tokenizer.pad_token = tokenizer.eos_token` to avoid padding warnings.
             - RETURN `(tokenizer, model, name, dtype_label)`
           except Exception as e:
             - Collect the error and continue to the next candidate.

      4) If all candidates fail:
         - Raise `RuntimeError` with a concise message and include the first/last error
           to aid debugging.

    Notes:
      - You may pass `low_cpu_mem_usage=True` or `device_map="auto"` (if appropriate for your runtime).
      - Keep `trust_remote_code=False` unless you explicitly need custom modeling code.
    """
    
    # Resolve model type
    torch_dtype, normalized_dtype = _resolve_dtype(dtype_str, DEVICE)

    errors = []
    # Try model candidates in order
    for mname in model_names:
        try: 
            # Get Tokenizer
            tokenizer = AutoTokenizer.from_pretrained(mname, use_fast=True)
            # Get Model
            model = AutoModelForCausalLM.from_pretrained(
                mname,
                torch_dtype=torch_dtype,
                trust_remote_code=False,
                low_cpu_mem_usage=True,
            )
            
            # Deploy mode to device
            model.to(DEVICE)
            
            # Put model in eval mode
            model.eval()

            # Set `tokenizer.pad_token = tokenizer.eos_token` to avoid padding warnings.
            if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
                tokenizer.pad_token = tokenizer.eos_token
                
            return tokenizer, model, mname, normalized_dtype
            
        except Exception as e:
            errors.append((mname, e))

    first_err = errors[0][1] if errors else "unknown"
    last_err = errors[-1][1] if errors else "unknown"

    raise RuntimeError(
        f"Failed to load any model from {model_names}. "
        f"First error: {first_err}. Last error: {last_err}."
    )

### Time Generate
Code Notes

#### 1)     inputs = tokenizer(prompt, return_tensors="pt").to(device)
- tokenizer returns an input dictionary of tensors
- to.device() deploys the input to the device (just like how model is deployed to device)
- both the model and inputs should be deployed to the same device otherwise it will error out

#### 2) torch.no_grad(): Warmup
```
    with torch.no_grad():
        for _ in range(num_warmup):
            output_ids = model_generate(input_ids, max_new_tokens = 8, use_cache = use_cache)
```
- in this you are running the inference loop , with reduced number of tokens = 8. You are running this multiple times
- **🎯 why torch.no_grad():** disables gradient checking. You are running inference, so no need for gradient checking unnecessary
   - Uses less memory
   - Runs faster
   - Prevents accidental training behavior
   - even if you forget to the torch.no_grad(). It will still run. But it will be slower and use more memory
- **🎯 why warmup :**
  - CUDA context setup: this itself takes 100-500ms
  - GPU kernel initialization: Many GPU operations select optimized kernels.Tune algorithms for your tensor shapes. Cache best execution strategy
  - Memory allocation: PyTorch builds its memory allocator pool. Allocates large blocks for future reuse
  - JIT kernel compilation
  - Cache building
- **🎯 why warmup multiple times :** usually 1 warmup is enough, but multiple warmups help based on the context
  - GPU frequency scaling: GPUs start in a lower power state. After a few runs, they boost to full clocks.
  - Memory pool stabilization: First few iterations may still trigger new allocations.
  - Caching behavior: Some internal optimizations settle after a few runs.
  - Variance smoothing: You want stable latency before measuring.
- **🎯 How many warmups do we need:** based on context
  - If you're benchmarking latency seriously: Use 5–10 warmup iterations.
  - If you're just running inference: You don’t need warmup at all.
  - If you're building a production API: One warmup call at startup is enough.Production cares about user latency, not benchmark purity
- **🎯 If use_cache=True, the first generation:**
   - KV Cache allocation: Allocates KV cache buffers. Sets up attention structures
   - KV cache stores past attention K and V (keys and values) so the model doesn’t recompute them every token. Only the new token’s Q/K/V is computed We don’t cache queries because queries are only used once
   - use_cache = True is used during inference only. If you're doing autoregressive text generation:👉 Always use use_cache=True unless you have a very specific reason not to.
  - use_cache NEVER used in TRAINING
  - without use_cache in Training, total complexity = O(n³). With use_cache = True, total complexity in training = O(n²)
- **🎯 torch.cuda.synchronize() during warmup**
   - I included it so that cpu and gpu are synced before the timed inferences begin.
   - But generally cuda.synchronize() is not needed during warmup
More eloquent answer from chatgpt on why synchronize after warmup is useful
   - Python may start queuing multiple generate calls in quick succession
   - GPU might still be doing the first warmup operation
   - First real inference might still see slightly higher latency because GPU is still initializing some things (from the warmup)

In [1]:
def time_generate(
    model,
    tokenizer,
    prompt: str,
    max_new_tokens: int = 50,
    use_cache: bool = True,
    num_warmup: int = 1,
    num_runs: int = 3,
) -> Dict[str, Any]:
        """
      1) Tokenize the prompt:
         - Create `input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(model.device)`.

      2) Warm up (no timing):
         - Under `torch.no_grad()`, run `model.generate(input_ids, max_new_tokens=8, use_cache=use_cache)`
           `num_warmup` times to stabilize kernels/clocks.
         - If CUDA, you may `torch.cuda.synchronize()` after warmup.

      3) Timed runs:
         - Initialize `latencies = []` and `tokens_out = []`.
         - Under `torch.no_grad()`, loop `num_runs` times:
             * If CUDA, `torch.cuda.synchronize()` before starting the timer.
             * Record `t0 = time.perf_counter()`.
             * Call `out = model.generate(input_ids, max_new_tokens=max_new_tokens, use_cache=use_cache)`.
             * If CUDA, `torch.cuda.synchronize()` to ensure all work has finished.
             * Record `t1 = time.perf_counter()` and append `(t1 - t0)` to `latencies`.
             * Compute generated token count: `gen = out.shape[-1] - input_ids.shape[-1]`
               and append to `tokens_out`.

      4) Aggregate metrics:
         - `L = average(latencies)` (seconds)
         - `TG = average(tokens_out)` (tokens)
         - `TPS = TG / L` (tokens/sec), guard divide-by-zero
         - `LPT = (L / max(TG, 1e-9)) * 1000.0` (ms/token)

      5) Return a dict with keys:
         - 'total_latency_s', 'tokens_generated', 'tokens_per_sec', 'avg_latency_per_token_ms'

      Notes:
        - Ensure `model.eval()` was called beforehand.
        - Do all forwards under `torch.no_grad()` to avoid autograd overhead.
        - We measure **end-to-end** generation time for the given `max_new_tokens`.
    """
    # 1) Tokenize the prompt:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 2) Warmup 
    with torch.no_grad():
        for _ in range(num_warmup):
            output_ids = model_generate(input_ids, max_new_tokens = 8, use_cache = use_cache)
        if device.type == "cuda":
            torch.cuda.synchronize()

    # Time Runs
    

NameError: name 'Dict' is not defined

In [ ]:
def time_generate(
    model,
    tokenizer,
    prompt: str,
    max_new_tokens: int = 50,
    use_cache: bool = True,
    num_warmup: int = 1,
    num_runs: int = 3,
) -> Dict[str, Any]:
    """
      1) Tokenize the prompt:
         - Create `input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(model.device)`.

      2) Warm up (no timing):
         - Under `torch.no_grad()`, run `model.generate(input_ids, max_new_tokens=8, use_cache=use_cache)`
           `num_warmup` times to stabilize kernels/clocks.
         - If CUDA, you may `torch.cuda.synchronize()` after warmup.

      3) Timed runs:
         - Initialize `latencies = []` and `tokens_out = []`.
         - Under `torch.no_grad()`, loop `num_runs` times:
             * If CUDA, `torch.cuda.synchronize()` before starting the timer.
             * Record `t0 = time.perf_counter()`.
             * Call `out = model.generate(input_ids, max_new_tokens=max_new_tokens, use_cache=use_cache)`.
             * If CUDA, `torch.cuda.synchronize()` to ensure all work has finished.
             * Record `t1 = time.perf_counter()` and append `(t1 - t0)` to `latencies`.
             * Compute generated token count: `gen = out.shape[-1] - input_ids.shape[-1]`
               and append to `tokens_out`.

      4) Aggregate metrics:
         - `L = average(latencies)` (seconds)
         - `TG = average(tokens_out)` (tokens)
         - `TPS = TG / L` (tokens/sec), guard divide-by-zero
         - `LPT = (L / max(TG, 1e-9)) * 1000.0` (ms/token)

      5) Return a dict with keys:
         - 'total_latency_s', 'tokens_generated', 'tokens_per_sec', 'avg_latency_per_token_ms'

      Notes:
        - Ensure `model.eval()` was called beforehand.
        - Do all forwards under `torch.no_grad()` to avoid autograd overhead.
        - We measure **end-to-end** generation time for the given `max_new_tokens`.
    """
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(model.device)
    with torch.no_grad():
        for _ in range(num_warmup):
            _ = model.generate(input_ids, max_new_tokens=8, use_cache=use_cache)
    latencies, tokens_out = [], []
    with torch.no_grad():
        for _ in range(num_runs):
            t0 = time.perf_counter()
            out = model.generate(input_ids, max_new_tokens=max_new_tokens, use_cache=use_cache)
            t1 = time.perf_counter()
            latencies.append(t1 - t0)
            tokens_out.append(out.shape[-1] - input_ids.shape[-1])
    L = sum(latencies) / max(len(latencies), 1)
    TG = sum(tokens_out) / max(len(tokens_out), 1)
    TPS = (TG / L) if L > 0 else float('nan')
    LPT = (L / max(TG, 1e-9)) * 1000.0
    return {'total_latency_s': L, 'tokens_generated': TG, 'tokens_per_sec': TPS, 'avg_latency_per_token_ms': LPT}


In [ ]:
def plot_xy(xs: List[float], ys: List[float], xlabel: str, ylabel: str, title: str):
    plt.figure()
    plt.plot(xs, ys, marker='o')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)
    plt.show()

## 3) Baseline
Run one fixed setup as baseline: `max_new_tokens=50`, `float16`, `use_cache=True`.

In [ ]:
tokenizer, model, MODEL_USED, DTYPE_USED = load_model_and_tokenizer(PREFERRED_MODEL, dtype_str='float16')
print(f"Loaded: {MODEL_USED} | dtype={DTYPE_USED}")
baseline = time_generate(model, tokenizer, DEFAULT_PROMPT, max_new_tokens=50, use_cache=True)
assert all(k in baseline for k in ['total_latency_s','tokens_generated','tokens_per_sec','avg_latency_per_token_ms'])
baseline

## 4) Exercise A — Generation Length vs Cost
Vary `max_new_tokens` and plot latency/throughput.

In [ ]:
def run_for_varied_lengths():
    """
    TODO:
      1) Define a list of generation lengths, e.g. lengths = [10, 50, 100, 250].

      2) For each L in lengths:
         - Call time_generate(model, tokenizer, DEFAULT_PROMPT, max_new_tokens=L, use_cache=True)
         - Collect each result dict in results_len.

      3) From results_len compute:
         - latencies = [r['total_latency_s'] for r in results_len]
         - throughputs = [r['tokens_per_sec'] for r in results_len]

      4) Plot two charts using plot_xy (one figure per chart):
         - Latency vs Generation Length
         - Throughput vs Generation Length
         (Use simple line+marker plots with grid; no custom colors.)

      5) (Optional) return a dict for downstream use:
         return {'lengths': lengths, 'latencies': latencies, 'throughputs': throughputs}
    """
    raise NotImplementedError("Implement run_for_varied_lengths() per the TODO above.")


run_for_varied_lengths()

## 5) Exercise B — Numerical Precision (float16 vs float32)
Reload for each precision with identical settings and compare.

In [ ]:
def run_varied_precision():
    """
    TODO:
      1) Define the list of precisions to test, e.g. ['float16', 'float32'].

      2) For each precision p in the list:
         - Load model + tokenizer using load_model_and_tokenizer(PREFERRED_MODEL, dtype_str=p).
         - Run time_generate with:
             * DEFAULT_PROMPT
             * max_new_tokens=50
             * use_cache=True
         - Store the result dictionary in compare_precision[p].

      3) Return compare_precision, which maps precision → generation performance metrics.

      4) (Optional) Plot or print results for easier comparison.
         Example metrics: latency, tokens/sec, latency per token.
    """
    raise NotImplementedError("Implement run_varied_precision() per the TODO above.")



compare_precision = run_varied_precision()

## 6) Exercise C — KV Cache On vs Off
Use the same model and toggle `use_cache`.

In [ ]:
kv_results = {}
kv_results['use_cache=True'] = time_generate(model, tokenizer, DEFAULT_PROMPT, max_new_tokens=50, use_cache=True)
kv_results['use_cache=False'] = time_generate(model, tokenizer, DEFAULT_PROMPT, max_new_tokens=50, use_cache=False)
kv_results

## 7) (Optional) Compact summary table
Summarize the three ideas in a small table.

In [ ]:
import pandas as pd
rows = []
rows.append({'label':'baseline (fp16, cache on)','dtype':'float16','use_cache':True,'max_new_tokens':50, **baseline})
rows.append({'label':'fp32, cache on','dtype':'float32','use_cache':True,'max_new_tokens':50, **compare_precision['float32']})
rows.append({'label':'fp16, cache off','dtype':'float16','use_cache':False,'max_new_tokens':50, **kv_results['use_cache=False']})
df = pd.DataFrame(rows)
df